# GTDBTk tree

In [1]:
import pandas as pd
from pathlib import Path
import yaml

import warnings
warnings.filterwarnings('ignore')

%load_ext rpy2.ipython

## File configurations

In [2]:
with open("config.yaml", "r") as f:
    notebook_configuration = yaml.safe_load(f)
notebook_configuration

{'bgcflow_dir': '/datadrive/bgcflow'}

In [14]:
# Set up paths and input file
bgcflow_dir = Path(notebook_configuration["bgcflow_dir"])
project_name = "mq_strepto"

tree_file = "/datadrive/bgcflow/data/processed/mq_strepto/gtdbtk_tree/gtdbtk.bac120.rooted.itol.tree"

In [17]:
%%R  -w 1200 -h 1200 -i R_tree
tree <- read.tree(R_tree)

In [35]:
R_tree

'/datadrive/bgcflow/data/processed/mq_strepto/gtdbtk_tree/gtdbtk.bac120.rooted.itol.tree'

In [36]:
%%R -i R_tree

# Load your tree from the Newick file
tree <- read.tree(R_tree)

# To get the distance matrix
distmat <- cophenetic(tree)

# To determine groups/clusters from the distance matrix, you can use hclust and cutree
# Here 0.5 is the threshold you choose to define your groups. You can replace it with your desired threshold
hclust_object <- hclust(distmat)
groups <- cutree(hclust_object, h = 0.5)

# # print the groups
print(groups)

R[write to console]: Error in if (is.na(n) || n > 65536L) stop("size cannot be NA nor exceed 65536") : 
  missing value where TRUE/FALSE needed




Error in if (is.na(n) || n > 65536L) stop("size cannot be NA nor exceed 65536") : 
  missing value where TRUE/FALSE needed


RInterpreterError: Failed to parse and evaluate line '\n# Load your tree from the Newick file\ntree <- read.tree(R_tree)\n\n# To get the distance matrix\ndistmat <- cophenetic(tree)\n\n# To determine groups/clusters from the distance matrix, you can use hclust and cutree\n# Here 0.5 is the threshold you choose to define your groups. You can replace it with your desired threshold\nhclust_object <- hclust(distmat)\ngroups <- cutree(hclust_object, h = 0.5)\n\n# # print the groups\nprint(groups)\n'.
R error message: 'Error in if (is.na(n) || n > 65536L) stop("size cannot be NA nor exceed 65536") : \n  missing value where TRUE/FALSE needed'

In [30]:
%%R
print(hclust_object)

R[write to console]: Error in print(hclust_object) : object 'hclust_object' not found




Error in print(hclust_object) : object 'hclust_object' not found


RInterpreterError: Failed to parse and evaluate line 'print(hclust_object)\n'.
R error message: "Error in print(hclust_object) : object 'hclust_object' not found"

In [31]:
%%R  -w 600 -h 1000

# Create color vector
colors = rainbow(max(groups)) # or choose other color palette you prefer
group_colors = colors[groups]

# Plot tree with colorized groups
plot(tree, tip.color = group_colors)

# Add threshold line
abline(h = 0.5, col = "red") # Change 'h' value to your actual threshold value


R[write to console]: Error in max(groups) : invalid 'type' (closure) of argument




Error in max(groups) : invalid 'type' (closure) of argument


RInterpreterError: Failed to parse and evaluate line '\n# Create color vector\ncolors = rainbow(max(groups)) # or choose other color palette you prefer\ngroup_colors = colors[groups]\n\n# Plot tree with colorized groups\nplot(tree, tip.color = group_colors)\n\n# Add threshold line\nabline(h = 0.5, col = "red") # Change \'h\' value to your actual threshold value\n'.
R error message: "Error in max(groups) : invalid 'type' (closure) of argument"

In [ ]:
%%R
p2 = p + geom_tiplab(aes(label=tip_label, fill=phylogroup),
                     size=2.6, hjust=-0.03, family='sans',
                    linetype = "dotted", linesize = 1) + # size of label border) +
        hexpand(1.5) +
        geom_tippoint(size=2.6, alpha=0.8, aes(colour=phylogroup, stroke=1, shape=sequence_quality)) + 
        scale_color_manual(labels=c("P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8"),
                           values=c('#264653','#e9c46a','#808080', '#808080','#f4a261','#808080', '#e76f51', '#2a9d8f'))
#p2

In [ ]:
%%R -i R_bgc_data
# add antismash_data
bgc_data <- read.csv(R_bgc_data)
p3 = p2 + new_scale_fill() + geom_facet(panel='BGCs Status',
                     geom=geom_bar,
                     data=bgc_data,
                     mapping=aes(x=value, fill=bgc_status),
                     position="stack",
                     orientation="y", 
                     stat="identity") +
        scale_fill_manual(labels=c("bgcs_on_contig_edge", "complete_bgcs"),
                          values=c("#cc2936", "#08415c")) 
#p3

In [ ]:
%%R -w 800 -h 500 -u px -i R_bigscape_data
# add bigscape_data
bigscape_data <- read.csv(R_bigscape_data)
p4 = p3 + new_scale_fill() + geom_facet(panel='BiG-SCAPE Class',
                     geom=geom_bar,
                     data=bigscape_data,
                     mapping=aes(x=value, fill=bigscape_class),
                     position="stack",
                     orientation="y", 
                     stat="identity") +
        scale_fill_manual(labels=c("NRPS", "Others", "PKS-NRP Hybrids", "PKSI", 
                                   "PKSOther", "RiPPs", "Saccharides", "Terpene"),
                          values=c("#734f5a", "#264653", "#2a9d8f", "#e9c46a", 
                                   "#f4a261", "#e76f51", "#941c2f", "#c05761")) 

p5 = p4 + theme(text=element_text(size=10, family="sans")) + theme_bw() 

p6 = facet_widths(p5, widths = c(80, 20, 30))
p6

In [ ]:
R_fig3c_svg = f"assets/figures/{FIGURE}/c.svg"
R_fig3c_pdf = f"assets/figures/{FIGURE}/c.pdf"

In [ ]:
%%R -i R_fig3c_svg -i R_fig3c_pdf
ggsave(file=R_fig3c_svg , plot=p6, device=svglite, width=3600, height=1800, units="px")
ggsave(plot=p6, width=3200, height=1600, units="px", dpi=300, filename=R_fig3c_pdf, useDingbats=FALSE)

## References
<font size="2">
{% for i in project().rule_used['automlst-wrapper']['references'] %}
- *{{ i }}*
{% endfor %}
</font>